In [200]:
from __future__ import annotations

import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [201]:
from pathlib import Path


from wufam.data.prepare_data import read_kf_data
from wufam.dataset import Dataset

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_25_M
FACTORS_DATASET = Dataset.FACTORS_M
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 12

In [202]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [203]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

In [204]:
import pandas as pd

mkt_caps = pd.read_csv(
    PATH / DATASET,
    skiprows=3_795,
    skipfooter=8881 - 4_984,
    index_col=0,
    engine="python",
)
mkt_caps.index = pd.to_datetime(mkt_caps.index, format="%Y%m")
mkt_caps = mkt_caps.loc[START:END]
mkt_caps.shape

(660, 25)

In [205]:
bms = pd.read_csv(
    PATH / DATASET,
    skiprows=4_991,
    skipfooter=8881 - 6_181,
    index_col=0,
    engine="python",
)
bms.index = pd.to_datetime(bms.index, format="%Y%m")
bms = bms.loc[START:END]
bms.shape

(660, 25)

In [206]:
ops = pd.read_csv(
    PATH / DATASET,
    skiprows=7_382,
    skipfooter=8881 - 8_128,
    index_col=0,
    engine="python",
)
ops.index = pd.to_datetime(ops.index, format="%Y%m")
ops = ops.loc[START:END]
ops.shape

(660, 25)

In [207]:
invs = pd.read_csv(
    PATH / DATASET,
    skiprows=8_133,
    skipfooter=8881 - 8_878,
    index_col=0,
    engine="python",
)
invs.index = pd.to_datetime(invs.index, format="%Y%m")
invs = invs.loc[START:END]
invs.shape

(660, 25)

In [208]:
lagged_ret = portfolios_xs_r.shift(1).fillna(0)
lagged_ret.shape

(660, 25)

In [209]:
stds = portfolios_xs_r.rolling(window=21, min_periods=1).std().fillna(0)
stds.shape

(660, 25)

In [210]:
skews = portfolios_xs_r.rolling(window=21, min_periods=1).skew().fillna(0)
skews.shape

(660, 25)

In [211]:
dfs_dict = {
    "Market_Caps": mkt_caps.resample("ME").last(),
    "Book_to_Market": bms.resample("ME").last(),
    "Operating_Profitability": ops.resample("ME").last(),
    "Investment": invs.resample("ME").last(),
    "Volatility": stds.resample("ME").last(),
    "Skewness": skews.resample("ME").last(),
    "Momentum": lagged_ret.resample("ME").last(),
    "ret": portfolios_xs_r.resample("ME").last().shift(-1).fillna(0),
}

# Stack each DataFrame to convert columns to index level, then concatenate
stacked_dfs = []
for name, df in dfs_dict.items():
    stacked = df.iloc[2:].stack()
    stacked.name = name
    stacked_dfs.append(stacked)

# Concatenate all stacked series along axis=1 (columns)
multi_df = pd.concat(stacked_dfs, axis=1)

# Set proper index names
multi_df.index.names = ["date", "portfolio"]
multi_df.shape

(16450, 8)

In [212]:
rank = (
    multi_df[multi_df.columns.difference(["ret"])]
    .groupby(level="date")
    .rank(ascending=False, method="max")
) + 1
rank = rank.div(rank.groupby(level="date").count()) - 0.5
rank = rank.fillna(0)

In [213]:
split_date = portfolios_xs_r.index[len(portfolios_xs_r) // 2]

In [214]:
tgt = multi_df["ret"]

In [215]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import RidgeCV

# lr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=12)
# lr = LinearRegression()
lr = RidgeCV(alphas=np.logspace(-3, 3, 100), cv=TimeSeriesSplit(n_splits=5))
lr.fit(rank.loc[:split_date], tgt.loc[:split_date])
pred = lr.predict(rank.loc[split_date:])
y_test = tgt.loc[split_date:]

In [216]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, pred)

0.0037510621059184137

In [217]:
mean_squared_error(y_test, tgt.loc[:split_date].mean() * np.ones(len(y_test)))

0.0037531412856691903

In [218]:
1 - mean_squared_error(y_test, pred) / mean_squared_error(
    y_test, tgt.loc[:split_date].mean() * np.ones(len(y_test))
)

0.0005539838744456071

In [219]:
from wufam.ap.ipca_factor_model import IPCAFactorModel

ipca = IPCAFactorModel(n_factors=5, fit_alpha=True)
ipca.fit(test_assets_xs_r=multi_df["ret"], ranks=rank)
ipca.predict(rank)

The panel dimensions are:
n_samples: 658 , L: 7 , T: 25
Step 1 - Aggregate Update: 1.064870625488357
Step 2 - Aggregate Update: 1.4961094804673802
Step 3 - Aggregate Update: 0.228600383983648
Step 4 - Aggregate Update: 1.8728601560331004
Step 5 - Aggregate Update: 1.677122301084553
Step 6 - Aggregate Update: 0.15104321400695522
Step 7 - Aggregate Update: 1.8495209236083736
Step 8 - Aggregate Update: 1.3936620235443313
Step 9 - Aggregate Update: 0.050012450808986864
Step 10 - Aggregate Update: 0.025181068611629775
Step 11 - Aggregate Update: 0.01382871717113654
Step 12 - Aggregate Update: 0.00908835120820961
Step 13 - Aggregate Update: 0.005980683423749911
Step 14 - Aggregate Update: 0.003945429406938761
Step 15 - Aggregate Update: 0.0026107563914381515
Step 16 - Aggregate Update: 0.001733374367347329
Step 17 - Aggregate Update: 0.0011548597770741909
Step 18 - Aggregate Update: 0.0007721330866923282
Step 19 - Aggregate Update: 0.0005180476156507741
Step 20 - Aggregate Update: 0.00034876

portfolio,BIG HiBM,BIG LoBM,ME1 BM2,ME1 BM3,ME1 BM4,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,ME5 BM2,ME5 BM3,ME5 BM4,SMALL HiBM,SMALL LoBM
date,,,,,,,,,,,,,,,,,,,,,
1970-03-31,0.005002,0.002059,0.016639,-0.005471,0.008129,0.008592,0.001569,-0.004605,0.011373,0.000202,...,0.009041,-0.003944,0.013614,0.014177,0.006769,0.000304,-0.003457,-0.004939,0.001273,-0.017263
1970-04-30,0.000314,0.006306,0.013600,0.002512,0.005693,0.004720,-0.001213,-0.000176,0.011013,0.006185,...,0.005977,0.000314,0.009225,0.004662,0.007878,0.006172,-0.000506,0.005215,0.004606,0.008316
1970-05-31,-0.000653,0.006455,0.013422,0.001182,0.008412,0.000116,0.001319,-0.000478,0.009758,0.007203,...,0.009224,-0.000640,0.007747,0.005395,0.007219,0.006780,-0.000712,0.005662,0.005977,0.006409
1970-06-30,0.000467,0.006145,0.012601,0.000091,0.004653,0.000149,0.003007,-0.001991,0.009596,0.005941,...,0.005482,-0.001806,0.008903,0.005632,0.007988,0.007211,-0.000229,0.006291,0.002066,0.008471
1970-07-31,0.005324,0.008461,0.011963,0.000794,0.009274,0.001277,0.012017,0.007790,0.009212,0.010150,...,0.010852,0.002203,0.005714,0.014438,0.009002,0.009410,-0.000540,0.005541,0.000959,0.005721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.006786,0.004708,0.006878,0.017695,0.001498,0.008845,0.006231,0.010857,0.007824,-0.000410,...,0.008471,0.003119,0.009178,0.000745,0.003265,0.006273,0.008143,0.002650,0.016872,0.033413
2024-09-30,0.007082,0.008044,0.008554,0.010745,0.006937,0.007651,0.006712,0.011942,0.008769,0.003044,...,0.015827,0.004431,0.012816,0.003479,0.002361,0.008507,0.008526,0.006030,0.014417,0.026404
2024-10-31,0.008413,0.007837,0.007260,0.009262,0.007499,0.009138,0.006054,0.009958,0.007367,0.002398,...,0.017143,0.004592,0.009133,0.007328,0.004120,0.007876,0.009020,0.000143,0.014495,0.032953


In [220]:
ret_true = pd.pivot_table(
    multi_df[["ret"]], index="date", columns="portfolio", values="ret"
)
ret_true.shape

(658, 25)

In [221]:
ipca.r2_score(ret_true, rank)

np.float64(0.9482219333318201)

In [222]:
split_date = portfolios_xs_r.index[len(portfolios_xs_r) // 2]

In [223]:
ipca = IPCAFactorModel(n_factors=5, fit_alpha=False)
ipca.fit(
    test_assets_xs_r=multi_df["ret"].loc[:split_date, :],
    ranks=rank.loc[:split_date, :, :],
)
ret_true = pd.pivot_table(
    multi_df[["ret"]].loc[split_date:, :],
    index="date",
    columns="portfolio",
    values="ret",
)

The panel dimensions are:
n_samples: 328 , L: 7 , T: 25
Step 1 - Aggregate Update: 1.2036359257318074
Step 2 - Aggregate Update: 0.12326237444377997
Step 3 - Aggregate Update: 0.07951670001143418
Step 4 - Aggregate Update: 0.07477977741103753
Step 5 - Aggregate Update: 0.07425769717675501
Step 6 - Aggregate Update: 0.07506388007850183
Step 7 - Aggregate Update: 0.0780715536868948
Step 8 - Aggregate Update: 0.08244538092316461
Step 9 - Aggregate Update: 0.08642565891399578
Step 10 - Aggregate Update: 1.7075066539766397
Step 11 - Aggregate Update: 0.0832937897892716
Step 12 - Aggregate Update: 0.11463965114124841
Step 13 - Aggregate Update: 0.27926290614265314
Step 14 - Aggregate Update: 1.2219840643723514
Step 15 - Aggregate Update: 0.22186501485774063
Step 16 - Aggregate Update: 1.1702713003748375
Step 17 - Aggregate Update: 0.13685047631947875
Step 18 - Aggregate Update: 0.2446304467595632
Step 19 - Aggregate Update: 1.5862878203303477
Step 20 - Aggregate Update: 0.09253249549381204
S

In [224]:
ipca.r2_score(ret_true, rank.loc[split_date:, :])

np.float64(-18.169304455836592)

In [225]:
ipca.r2_gls_score(ret_true, rank.loc[split_date:, :])

np.float64(-21.76164334739709)

In [226]:
ipca_total_r = ipca.get_mv_weights(ret_true, rank.loc[split_date:, :])

In [227]:
from wufam.metrics.metrics import calc_sharpe

calc_sharpe(
    strategy_total_r=ipca_total_r,
    rf_rate=rf,
    factor_annualize=FACTOR_ANNUALIZE,
)

-13.7045109887146